In [1]:
import h5py

import numpy as np
import awkward as ak

import hepinfo.models
from hepinfo.util import readFromAnomalyBackgroundh5, awkward_to_numpy

from qkeras import quantized_bits

ModuleNotFoundError: No module named 'hepinfo.models'

In [10]:
# prepare dataset for inference
NEG=4
NMU=4
NJET=10

# To load BSM signal files, for now look at data only
# process="GluGluHToBB_M-125"
# X_signal =readFromAnomalySignalh5('2024E-BSM.h5', process=process, object_ranges='default2') #to load signal
X = readFromAnomalyBackgroundh5('run370293.h5', object_ranges='default2')

X_flat = np.zeros((X[0]['nEvents'], 57), dtype='int')
X_flat[:,0] = ak.flatten(X[2].sums[:].pt)
X_flat[:,2] = ak.flatten(X[2].sums[:].phi)
X_flat[:,3:3+3*(NEG)] = awkward_to_numpy(X[2].egs, NEG)
X_flat[:,3+3*(NEG):3+3*(NMU+NEG)] = awkward_to_numpy(X[2].muons, NMU)
X_flat[:,3+3*(NMU+NEG):3+3*(NMU+NEG+NJET)] = awkward_to_numpy(X[2].jets, NJET)

NameError: name 'readFromAnomalyBackgroundh5' is not defined

In [ ]:
 # load model and scaling parameters
input_quantizer = quantized_bits(8,5,alpha=1)
scales = h5py.File('scales.h5')
scale_data = scales['norm_scale'][:].flatten()
offset_data = scales['norm_bias'][:].flatten()
nPV = ak.to_numpy(X[1]['nPV'])
nPV_Good = ak.to_numpy(X[1]['nPV_Good'])
X_scaled = input_quantizer((X_flat.astype('float') - offset_data) / scale_data)
batch_size = len(X_scaled)